In [1]:
from utils import time_op
import impmeas as imp

solver = imp.formulas.GPMC()
imp.formulas.set_pmc_solver(solver)

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

imp.formulas.set_buddy_context(vars)
f = time_op(imp.BuddyNode.parse)(formula)
satcount_bdd = time_op(f.expectation)() * 2**n
print(f"BDD satcount = {satcount_bdd}")

[00006.9296 ms / 0.0069 s / 0.0001 min] satcount
GPMC satcount = 246
[00078.0425 ms / 0.0780 s / 0.0013 min] parse
[00000.0093 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 246.0


In [3]:
n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 
imp.formulas.set_buddy_context(vars)

x = "x1"
f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, alternative=True, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00547.1184 ms / 0.5471 s / 0.0091 min] parse
bdd size = 73
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7fbe09b53f10> ===
k=0 d result=0.3066 max increase possible=0.3467 
k=1 d result=0.3018 max increase possible=0.0299 
k=2 d result=0.0299 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.6383, can only be increased by 0.0000.
=== DONE ===
[00017.2114 ms / 0.0172 s / 0.0003 min] blame
[00000.0432 ms / 0.0000 s / 0.0000 min] influence
influence 0.30664, 0.63835 <= blame


In [5]:
x = "x1"
f = time_op(imp.Formula.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, alternative=False, debug=True)
infl = (time_op)(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00607.5187 ms / 0.6075 s / 0.0101 min] parse
=== COMPUTING BLAME for x1 in Formula with size 1752 ===


In [6]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)
imp.formulas.set_buddy_context(vars)

f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00771.8520 ms / 0.7719 s / 0.0129 min] parse
[01760.7563 ms / 1.7608 s / 0.0293 min] blame
0.57487 <= blame
[00012.7566 ms / 0.0128 s / 0.0002 min] influence
influence = 0.26758
[06292.9702 ms / 6.2930 s / 0.1049 min] <lambda>
hammer = 0.06516
[00824.3942 ms / 0.8244 s / 0.0137 min] dominating_cgm
[00607.8038 ms / 0.6078 s / 0.0101 min] rectifying_cgm
[00664.0453 ms / 0.6640 s / 0.0111 min] parse
bdd size = 90
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f53ec72e770> ===
k=0 d result=0.2676 max increase possible=0.3662 
k=1 d result=0.2046 max increase possible=0.1077 
k=2 d result=0.0885 max increase possible=0.0144 
k=3 d result=0.0132 max increase possible=0.0010 
k=4 d result=0.0010 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5749, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00175.9050 ms / 0.1759 s / 0.0029 min] blame
[00000.0613 ms / 0.0001 s / 0.0000 min] influence
influence 0.26758,

: 